In [3]:
pwd

'C:\\Users\\82102\\Desktop\\nlp'

In [1]:
# gpu 잡히는지 확인
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8061152870070770646, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14474280960
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9273431993164810406
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:1e.0, compute capability: 7.5"]

# 1. pipeline 이용해서

In [2]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering, QuestionAnsweringPipeline
import tensorflow as tf
import tqdm.notebook
import math

In [3]:
model = TFAutoModelForQuestionAnswering.from_pretrained("./Untitled Folder", from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("./Untitled Folder")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertForQuestionAnswering: ['sop_classifier.classifier.weight', 'sop_classifier.classifier.bias', 'albert.embeddings.position_ids']
- This IS expected if you are initializing TFAlbertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFAlbertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# QA pipeline 세팅
qa = QuestionAnsweringPipeline(model=model, tokenizer=tokenizer)

In [5]:
context = '윤여정은 1966년 연극 배우로 연기 경력을 시작하였고, 2021년 영화 《미나리》의 순자 역으로 아카데미 여우조연상을 수상했다.'

In [6]:
question = '윤여정의 직업은?'

In [7]:
qa(question=question, context=context)

{'score': 0.0009075522539205849, 'start': 32, 'end': 40, 'answer': '2021년 영화'}

In [8]:
qa(question='윤여정은 2021년 무엇을 했나?', context=context)

{'score': 0.001071204082109034, 'start': 32, 'end': 40, 'answer': '2021년 영화'}

In [9]:
qa(question='윤여정의 미나리에서의 역할은?', context=context)

{'score': 0.0010481227654963732, 'start': 32, 'end': 40, 'answer': '2021년 영화'}

# 2. 한국어 QA fine-tunning

In [11]:
model = TFAutoModelForQuestionAnswering.from_pretrained("./Untitled Folder", from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("./Untitled Folder")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertForQuestionAnswering: ['sop_classifier.classifier.weight', 'sop_classifier.classifier.bias', 'albert.embeddings.position_ids']
- This IS expected if you are initializing TFAlbertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFAlbertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 데이터(KorQUAD) 전처리


In [1]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering, QuestionAnsweringPipeline
import tensorflow as tf
import urllib.request
import json

In [2]:
# korquad 파일 다운로드
urllib.request.urlretrieve('https://korquad.github.io/dataset/KorQuAD_v1.0_train.json', 'korquad.json')

('korquad.json', <http.client.HTTPMessage at 0x1e2901d0490>)

In [3]:
korquad = json.load(open('korquad.json', encoding='utf8'))

In [4]:
korquad['data'][0]

{'paragraphs': [{'qas': [{'answers': [{'text': '교향곡', 'answer_start': 54}],
     'id': '6566495-0-0',
     'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'},
    {'answers': [{'text': '1악장', 'answer_start': 421}],
     'id': '6566495-0-1',
     'question': '바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?'},
    {'answers': [{'text': '베토벤의 교향곡 9번', 'answer_start': 194}],
     'id': '6566495-0-2',
     'question': '바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?'},
    {'answers': [{'text': '파우스트', 'answer_start': 15}],
     'id': '6566518-0-0',
     'question': '1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?'},
    {'answers': [{'text': '합창교향곡', 'answer_start': 354}],
     'id': '6566518-0-1',
     'question': '파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?'},
    {'answers': [{'text': '1839', 'answer_start': 0}],
     'id': '5917067-0-0',
     'question': '바그너가 파우스트를 처음으로 읽은 년도는?'},
    {'answers': [{'text': '파리', 'answer_start': 410}],
     'id': '5917067-0-1',
     'question': '바그너가 처음 교향곡 작곡을 한 장소는?'},
    {'answers': [{'text': '드레스덴', 'answer_sta

In [9]:
para = korquad['data'][0]['paragraphs'][0]
para

{'qas': [{'answers': [{'text': '교향곡', 'answer_start': 54}],
   'id': '6566495-0-0',
   'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'},
  {'answers': [{'text': '1악장', 'answer_start': 421}],
   'id': '6566495-0-1',
   'question': '바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?'},
  {'answers': [{'text': '베토벤의 교향곡 9번', 'answer_start': 194}],
   'id': '6566495-0-2',
   'question': '바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?'},
  {'answers': [{'text': '파우스트', 'answer_start': 15}],
   'id': '6566518-0-0',
   'question': '1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?'},
  {'answers': [{'text': '합창교향곡', 'answer_start': 354}],
   'id': '6566518-0-1',
   'question': '파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?'},
  {'answers': [{'text': '1839', 'answer_start': 0}],
   'id': '5917067-0-0',
   'question': '바그너가 파우스트를 처음으로 읽은 년도는?'},
  {'answers': [{'text': '파리', 'answer_start': 410}],
   'id': '5917067-0-1',
   'question': '바그너가 처음 교향곡 작곡을 한 장소는?'},
  {'answers': [{'text': '드레스덴', 'answer_start': 534}],
   'id': '5917067-0-2',
   'question': '바그너의 1

In [6]:
# 지문
para['context']

'1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.'

In [7]:
# 질문과 답
qas = para['qas'][0]
qas

{'answers': [{'text': '교향곡', 'answer_start': 54}],
 'id': '6566495-0-0',
 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'}

In [8]:
inputs = tokenizer(para['context'], 
                   qas['question'], 
                   truncation=True, 
                   max_length=tokenizer.model_max_length, 
                   return_tensors='tf')


NameError: name 'tokenizer' is not defined

In [20]:
q = qas['answers'][0]

# 글자의 위치
start_char = q['answer_start']
end_char = start_char + len(q['text']) - 1

# 토큰의 위치
start = inputs.char_to_token(0, start_char)
end = inputs.char_to_token(0, end_char)

In [21]:
# 글자의 위치
print(start_char)
print(end_char)

54
56


In [22]:
# 토큰의 위치
print(start)
print(end)

41
43


In [23]:
inputs['input_ids']

<tf.Tensor: shape=(1, 512), dtype=int32, numpy=
array([[   2,   21, 6355, 6336, 6672, 6545, 4209, 6424, 6686, 6259, 3484,
        6518, 6362, 5283, 6349, 6350, 6419, 6377, 5022, 6537, 4783, 6322,
        3517, 3662, 6463, 6271, 4092, 6537, 6263, 3630, 6628, 4779, 6335,
        4445, 6608, 6354, 5381, 6369, 5369, 6351, 6362, 3489, 6719, 6342,
        6377, 4574, 6628, 6259, 3965, 6377, 3401, 6259, 6265,   18, 4779,
        4514, 6341, 4209, 6424, 6686, 6259,   21, 6355, 6336, 6355, 6545,
        6271, 4316, 3831, 7283, 6353, 6354, 4380, 6410, 6394, 6410, 6377,
        3774,    1, 4388, 7025, 6263, 6422, 4861, 6513, 6257, 4518, 6588,
        6271, 3389, 6381, 6371, 6353, 6329, 4131, 6572, 6350, 6316, 7315,
        6517, 6350, 6335, 4094, 6351, 6259, 5283, 6349, 6350, 6419, 6362,
        4520, 6429, 6271, 3472, 6526, 6371, 6265, 6322, 5371, 6265,   18,
        3935, 6483, 5283, 6317, 6271, 6369, 4590, 6408, 6584, 6587, 6362,
        4908, 6969, 6354, 5283, 6317, 4769, 6445, 6363, 3477, 65

In [24]:
# 정답을 실제 텍스트로 변환
tokenizer.decode(inputs['input_ids'].numpy()[0, start:end+1])

'교향곡'

In [25]:
inputs['start_positions'] = [start]
inputs['end_positions'] = [end]

## TFRecord로 저장


In [26]:
def int_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def make_inputs(context, qas):
    inputs = tokenizer(
        context, 
        qas['question'], 
        truncation=True, 
        max_length=tokenizer.model_max_length)
    q = qas['answers'][0]
    start_char = q['answer_start']
    end_char = start_char + len(q['text']) - 1
    start = inputs.char_to_token(0, start_char)
    end = inputs.char_to_token(0, end_char)
    inputs['start_positions'] = [start]
    inputs['end_positions'] = [end]
    return inputs

In [27]:
%%time
n = 0
filename = 'korquad.tfrecord'
with tf.io.TFRecordWriter(filename) as writer:
    for item in korquad['data']:   # 각 아이템 순환
        for para in item['paragraphs']:  # 아이템마다 
            context = para['context']
            for qas in para['qas']:
                inputs = make_inputs(context, qas)
                if inputs['start_positions'][0] and inputs['end_positions'][0]:
                    feature = {k: int_feature(v) for k, v in inputs.items()}
                    example = tf.train.Example(features=tf.train.Features(feature=feature))
                    s = example.SerializeToString()
                    writer.write(s)
                    n += 1


CPU times: user 54.4 s, sys: 231 ms, total: 54.6 s
Wall time: 54.6 s


In [28]:
# 사례 수
n

58675

## 불러오기

In [29]:
int_seq = tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True)
int_value = tf.io.FixedLenFeature(shape=(), dtype=tf.int64)
feature_description = {
    'input_ids': int_seq,
    'token_type_ids': int_seq,
    'attention_mask': int_seq,
    'start_positions': int_value,
    'end_positions': int_value
}


In [30]:
def preproc(example):
    example = tf.io.parse_single_example(example, feature_description)
    example = {k : tf.cast(v, tf.int32) for k, v in example.items()}
    return example

In [31]:
# korquad 데이터셋 불러오기
dataset = tf.data.TFRecordDataset(['korquad.tfrecord']).map(preproc).padded_batch(8)
batch = next(iter(dataset))
result = model(batch)

## 수동으로 학습

In [32]:
%%time
# 옵티마이저는 adam
opt = tf.keras.optimizers.Adam(learning_rate=5e-5)

n = 57653
for batch in tqdm.notebook.tqdm(dataset, total=math.ceil(n / 32)):
    with tf.GradientTape() as tape:
        result = model(batch)
        loss = tf.reduce_mean(result['loss'])
    grads = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(grads, model.trainable_variables)) # loss가 감소하는 방향으로 파라미터 업데이트

  0%|          | 0/1802 [00:00<?, ?it/s]

CPU times: user 33min 15s, sys: 2min 22s, total: 35min 38s
Wall time: 1h 36min 23s


In [33]:
model

In [34]:
context = '윤여정은 1966년 연극 배우로 연기 경력을 시작하였고, 2021년 영화 《미나리》의 순자 역으로 아카데미 여우조연상을 수상했다.'

In [35]:
def qa(question, context):
    inputs = tokenizer(context, question, add_special_tokens=True, return_tensors="tf")
    outputs = model(inputs)
    start = tf.argmax(outputs.start_logits, axis=1).numpy()[0]
    end = tf.argmax(outputs.end_logits, axis=1).numpy()[0]
    return tokenizer.decode(inputs['input_ids'][0, start:end+1])

In [36]:
qa('윤여정은 언제 데뷔했나?', context)

'1966년'

In [37]:
qa('윤여정의 직업은?', context)

'연극 배우'

In [38]:
qa('윤여정이 2021년 받은 상은?', context)

'아카데미 여우조연상'

In [39]:
qa('윤여정의 출연작은?', context)

'미나리'

In [40]:
qa('윤여정이 미나리에서 맡았던 역할은?', context)

'순자'

In [41]:
qa('윤여정은 2021년 무엇을 했는가?', context)

'미나리'

In [42]:
qa('윤여정이 미나리에서 맡았던 배역은?', context)

'순자'

# 2. 한국어 QA fine-tunning

In [43]:
korquad = json.load(open('./Untitled Folder/financial_mrc_test.json', encoding='utf8'))

In [44]:
korquad['data'][0]

{'title': '(금융경제신문 문혜원 기자)신한은행은 지난 6일 ▲본부...',
 'url': None,
 'context': '(금융경제신문 문혜원 기자)신한은행은 지난 6일 ▲본부부서 조직개편과 ▲현장 영업동력 강화 및 커뮤니티 지원에 중점을 둔 하반기 정기인사를 실시했다고 밝혔다.이번 조직개편은 디지털과 글로벌을 향한 신한만의 새로운 길을 만들어 ‘초(超)격차의 리딩뱅크’와 ‘World Class Bank 신한’의 꿈을 실현하기 위한 위성호 은행장의 전략을 바탕으로 진행됐다.조직개편의 주요 내용은 ▲디지털그룹 ▲GIB그룹 ▲대기업그룹 ▲글로벌사업본부 신설이다. 디지털그룹은 기존 분산되어 있던 디지털 인적·물적 역량 및 사업전략을 총괄하는 조직으로 신한은행의 디지털전략을 총괄하는 디지털전략본부, 모바일 채널 통합 플랫폼 구축을 위한 디지털채널본부, 빅데이터 분석역량 강화를 위한 빅데이터센터로 구성된다.또한 유연한 디지털 조직 운영을 위해 디지털그룹 내 A.I, 블록체인 등 총 7개의 랩(Lab)조직을 신설했다. GIB그룹과 대기업그룹은 자본시장 경쟁력 강화를 위해 기존 CIB그룹을 분리 확대 개편하면서 신설됐다.GIB그룹은 ‘원 신한(One Shinhan)’ 관점의 그룹경쟁력 강화를 위한 그룹 매트릭스 조직으로 사업부문 소속 직원들의 협업을 통해 역량을 집중할 계획이다.글로벌사업본부는 글로벌시장 경쟁력 강화를 위해 글로벌사업그룹 내 신설된 조직으로 앞으로 신한은행의 글로벌영업과 전략을 담당하게 된다.조직개편과 함께 진행된 이번 하반기 정기 인사는 조직개편 지원과 현장 영업동력 강화를 키워드로 실시됐다.디지털그룹 신설에 맞춰 디지털 역량 강화를 위해 선발 교육한 20여명의 대리ž행원급 인력을 실무부서에 배치했다. 아울러 현장의 전문성 강화를 위해 기업금융 심화과정 대상자 40여명을 선발하고 하반기 집중적인 전문 교육과정을 통해 기업금융 전문가로 육성할 계획이다.이밖에도 커뮤니티 단위의 자체 인력운용계획을 사전에 취합해 현장 요청사항 및 지역별 특수성을 감

In [45]:
# 지문
para = korquad['data'][0]['context']
para

'(금융경제신문 문혜원 기자)신한은행은 지난 6일 ▲본부부서 조직개편과 ▲현장 영업동력 강화 및 커뮤니티 지원에 중점을 둔 하반기 정기인사를 실시했다고 밝혔다.이번 조직개편은 디지털과 글로벌을 향한 신한만의 새로운 길을 만들어 ‘초(超)격차의 리딩뱅크’와 ‘World Class Bank 신한’의 꿈을 실현하기 위한 위성호 은행장의 전략을 바탕으로 진행됐다.조직개편의 주요 내용은 ▲디지털그룹 ▲GIB그룹 ▲대기업그룹 ▲글로벌사업본부 신설이다. 디지털그룹은 기존 분산되어 있던 디지털 인적·물적 역량 및 사업전략을 총괄하는 조직으로 신한은행의 디지털전략을 총괄하는 디지털전략본부, 모바일 채널 통합 플랫폼 구축을 위한 디지털채널본부, 빅데이터 분석역량 강화를 위한 빅데이터센터로 구성된다.또한 유연한 디지털 조직 운영을 위해 디지털그룹 내 A.I, 블록체인 등 총 7개의 랩(Lab)조직을 신설했다. GIB그룹과 대기업그룹은 자본시장 경쟁력 강화를 위해 기존 CIB그룹을 분리 확대 개편하면서 신설됐다.GIB그룹은 ‘원 신한(One Shinhan)’ 관점의 그룹경쟁력 강화를 위한 그룹 매트릭스 조직으로 사업부문 소속 직원들의 협업을 통해 역량을 집중할 계획이다.글로벌사업본부는 글로벌시장 경쟁력 강화를 위해 글로벌사업그룹 내 신설된 조직으로 앞으로 신한은행의 글로벌영업과 전략을 담당하게 된다.조직개편과 함께 진행된 이번 하반기 정기 인사는 조직개편 지원과 현장 영업동력 강화를 키워드로 실시됐다.디지털그룹 신설에 맞춰 디지털 역량 강화를 위해 선발 교육한 20여명의 대리ž행원급 인력을 실무부서에 배치했다. 아울러 현장의 전문성 강화를 위해 기업금융 심화과정 대상자 40여명을 선발하고 하반기 집중적인 전문 교육과정을 통해 기업금융 전문가로 육성할 계획이다.이밖에도 커뮤니티 단위의 자체 인력운용계획을 사전에 취합해 현장 요청사항 및 지역별 특수성을 감안해 정기인사에 반영했다.신한은행 관계자는 “이번 조직개편과 인사이동은 치열해지는 금융환경에서 더 높이 비상하는 신한은행이 되기 위

In [46]:
# 질문과 답
qas = korquad['data'][0]['qas'][0]
qas

{'id': '0',
 'question': '위성호 은행장에 의해 개편된 신설조직 중, ‘원 신한(One Shinhan)’ 관점의 그룹경쟁력 강화를 위한 그룹 매트릭스 조직으로 사업부문 소속 직원들의 협업을 통해 역량을 집중할 계획을 가진 그룹명은?',
 'answer': {'text': 'GIB그룹',
  'answer_start': 504,
  'html_answer_text': 'GIB그룹',
  'html_answer_start': 504}}

In [47]:
qas['question']

'위성호 은행장에 의해 개편된 신설조직 중, ‘원 신한(One Shinhan)’ 관점의 그룹경쟁력 강화를 위한 그룹 매트릭스 조직으로 사업부문 소속 직원들의 협업을 통해 역량을 집중할 계획을 가진 그룹명은?'

In [48]:
inputs = tokenizer(para , 
                   qas['question'], 
                   truncation=True, 
                   max_length=tokenizer.model_max_length, 
                   return_tensors='tf')
inputs

{'input_ids': <tf.Tensor: shape=(1, 512), dtype=int32, numpy=
array([[   2,   12, 3523, 6784, 6429, 6258, 6444, 6282, 4168, 6637, 6363,
        3529, 6367,   13, 4516, 6483, 6296, 6603, 6296, 4908, 6477,   26,
        6531,  453, 6541, 6281, 6281, 6369, 4850, 6753, 6310, 6428, 6257,
         453, 6577, 6347, 4667, 6293, 6346, 6440, 3400, 6328, 4207, 5112,
        6978, 6426, 6327, 4908, 6363, 6271, 4884, 6451, 6377, 3858, 5369,
        6436, 6341, 4830, 6341, 6360, 6376, 6335, 4518, 6266, 6371, 6265,
        6322, 4215, 6967, 6265,   18, 4779, 6960, 4850, 6753, 6310, 6428,
        6296, 3886, 6361, 7075, 6257, 3521, 6354, 6619, 6377, 5393, 6483,
        4516, 6483, 6356, 6362, 4390, 6354, 6345, 3533, 6377, 4094, 6368,
        6318,  318, 5043,   12, 2987,   13, 3447, 6320, 6362, 4083, 7034,
        7546, 6587,  319, 4691,  318,   59, 6460, 6290, 6386, 6288,   39,
        6386, 6622, 6663, 6663,   38, 6622, 6461, 6287, 4516, 6483,  319,
        4776, 3608, 6377, 4518, 6577, 6298, 6341, 

In [49]:
q = qas['answer']

# 글자의 위치
start_char = q['answer_start']
end_char = start_char + len(q['text']) - 1

# 토큰의 위치
start = inputs.char_to_token(0, start_char)
end = inputs.char_to_token(0, end_char)

In [50]:
# 글자의 위치
print(start_char)
print(end_char)

504
508


In [51]:
# 토큰의 위치
print(start)
print(end)

404
408


In [52]:
inputs['input_ids']

<tf.Tensor: shape=(1, 512), dtype=int32, numpy=
array([[   2,   12, 3523, 6784, 6429, 6258, 6444, 6282, 4168, 6637, 6363,
        3529, 6367,   13, 4516, 6483, 6296, 6603, 6296, 4908, 6477,   26,
        6531,  453, 6541, 6281, 6281, 6369, 4850, 6753, 6310, 6428, 6257,
         453, 6577, 6347, 4667, 6293, 6346, 6440, 3400, 6328, 4207, 5112,
        6978, 6426, 6327, 4908, 6363, 6271, 4884, 6451, 6377, 3858, 5369,
        6436, 6341, 4830, 6341, 6360, 6376, 6335, 4518, 6266, 6371, 6265,
        6322, 4215, 6967, 6265,   18, 4779, 6960, 4850, 6753, 6310, 6428,
        6296, 3886, 6361, 7075, 6257, 3521, 6354, 6619, 6377, 5393, 6483,
        4516, 6483, 6356, 6362, 4390, 6354, 6345, 3533, 6377, 4094, 6368,
        6318,  318, 5043,   12, 2987,   13, 3447, 6320, 6362, 4083, 7034,
        7546, 6587,  319, 4691,  318,   59, 6460, 6290, 6386, 6288,   39,
        6386, 6622, 6663, 6663,   38, 6622, 6461, 6287, 4516, 6483,  319,
        4776, 3608, 6377, 4518, 6577, 6298, 6341, 4747, 6483, 47

In [53]:
# 정답을 실제 텍스트로 변환
tokenizer.decode(inputs['input_ids'].numpy()[0, start:end+1])

'GIB그룹'

In [54]:
inputs['start_positions'] = [start]
inputs['end_positions'] = [end]

In [55]:
def int_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def make_inputs(context, qas):
    inputs = tokenizer(
        context, 
        qas['question'], 
        truncation=True, 
        max_length=tokenizer.model_max_length)
    q = qas['answer']
    start_char = q['answer_start']
    end_char = start_char + len(q['text']) - 1
    start = inputs.char_to_token(0, start_char)
    end = inputs.char_to_token(0, end_char)
    inputs['start_positions'] = [start]
    inputs['end_positions'] = [end]
    return inputs

In [56]:
%%time
n = 0
filename = 'korquad.tfrecord1'
with tf.io.TFRecordWriter(filename) as writer:
    for item in korquad['data']:   # 각 아이템 순환
        context = item['context']  # 아이템마다 
            
        for qas in item['qas']:
            inputs = make_inputs(context, qas)
            if inputs['start_positions'][0] and inputs['end_positions'][0]:
                feature = {k: int_feature(v) for k, v in inputs.items()}
                example = tf.train.Example(features=tf.train.Features(feature=feature))
                s = example.SerializeToString()
                writer.write(s)
                n += 1


CPU times: user 555 ms, sys: 0 ns, total: 555 ms
Wall time: 554 ms


In [57]:
int_seq = tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True)
int_value = tf.io.FixedLenFeature(shape=(), dtype=tf.int64)
feature_description = {
    'input_ids': int_seq,
    'token_type_ids': int_seq,
    'attention_mask': int_seq,
    'start_positions': int_value,
    'end_positions': int_value
}


In [58]:
def preproc(example):
    example = tf.io.parse_single_example(example, feature_description)
    example = {k : tf.cast(v, tf.int32) for k, v in example.items()}
    return example

In [59]:
# korquad 데이터셋 불러오기
dataset = tf.data.TFRecordDataset(['korquad.tfrecord1']).map(preproc).padded_batch(8)
batch = next(iter(dataset))
result = model(batch)

In [60]:
%%time
# 옵티마이저는 adam
opt = tf.keras.optimizers.Adam(learning_rate=5e-5)

n = 333
for batch in tqdm.notebook.tqdm(dataset, total=math.ceil(n / 32)):
    with tf.GradientTape() as tape:
        result = model(batch)
        loss = tf.reduce_mean(result['loss'])
    grads = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(grads, model.trainable_variables)) # loss가 감소하는 방향으로 파라미터 업데이트

  0%|          | 0/11 [00:00<?, ?it/s]

CPU times: user 11.7 s, sys: 1.02 s, total: 12.7 s
Wall time: 36.2 s


In [61]:
model

In [71]:
context = '방카슈랑스는 금융의 겸업화 추세에 부응하여 금융산업의 선진화를 도모하고 금융소비자의 편익을 위하여 도입되었습니다.'

In [72]:
def qa(question, context):
    inputs = tokenizer(context, question, add_special_tokens=True, return_tensors="tf")
    outputs = model(inputs)
    start = tf.argmax(outputs.start_logits, axis=1).numpy()[0]
    end = tf.argmax(outputs.end_logits, axis=1).numpy()[0]
    return tokenizer.decode(inputs['input_ids'][0, start:end+1])

In [73]:
qa('금융의 겸업화 추세에 부응하여 금융산업의 선진화를 도모하고 금융소비자의 편익을 위하여 도입된 것은?', context)

'방카슈랑스'

In [74]:
context = '10일 국내 가상화폐 시장에서 비트코인이 급락하며 4천만원선까지 위협하고 있다.이날 오전 9시 35분 가상화폐 거래소 업비트에서 1비트코인 가격은 4천36만4천원으로, 24시간 만에 10.06% 내렸다. 비트코인이 4천만원까지 내려온 것은 지난해 5월 대폭락 직후인 7월 26일로부터 1년 9개월여 만의 처음이다.같은 시간 빗썸에서도 10.34% 하락하며 4천12만1천원에 거래되고 있다. 이는 최근 비트코인이 동조하는 뉴욕 증시가 간밤 스태그플레이션(고물가·저성장) 공포 때문에 크게 하락한 영향으로 풀이된다. 3대 지수 중 기술주 중심 나스닥 지수는 4.29% 떨어졌다. 52주 전 최고치와 대비하면 28%나 하락한 수준이다. 스탠더드앤드푸어스(S&P)500지수 역시 종가 기준 지난해 3월 31일 이후 처음으로 4,000선 아래로 내려갔다.이에 미국에서도 한때 비트코인 가격은 지난해 7월 말 이후 처음으로 3만달러 선 아래로 내려가기도 했다.국내 시가총액 규모 2위인 이더리움도 10%대로 하락하며 업비트에서 한때 296만8천원까지 떨어져 지난 2월 25일 이후 2개월여 만에 300만원선을 밑돌기도 했다.'

In [75]:
def qa(question, context):
    inputs = tokenizer(context, question, add_special_tokens=True, return_tensors="tf")
    outputs = model(inputs)
    start = tf.argmax(outputs.start_logits, axis=1).numpy()[0]
    end = tf.argmax(outputs.end_logits, axis=1).numpy()[0]
    return tokenizer.decode(inputs['input_ids'][0, start:end+1])

In [76]:
qa('비트코인은 어디까지 급락했나?', context)

'4천만원선'

In [78]:
qa('현재 기술주 중심 나스닥 지수는 얼마나 떨어졌나?', context)

'4. 29 %'